In [1]:
import pandas as pd
import os
import re
import numpy as np

# Display all columns when previewing dataframes
from IPython.display import display
pd.options.display.max_columns = None

In [2]:
# Change path to where spotify_top200.csv is located

# path = '/home/jovyan/work/Documents/Correlation One/spotipy/'
path = r'/Users/minhhoang/Documents/Correlation One/spotipy'
path_input = os.path.join(path, 'input')
# os.getcwd()

In [3]:
# data_top200 = pd.read_csv(os.path.join(path_input, 'Spotify_Daily_Streaming.csv'))
data_top200 = pd.read_csv(os.path.join(path_input, 'spotify_top200.csv'))

In [4]:
# data_top200.head()
print(len(data_top200.index))
data_top200['Country'] = data_top200['Country'].apply(lambda x: 'Viet Nam' if x == 'Viet Name' else x) 
data_top200['Date'] = pd.to_datetime(data_top200['Date'])

14546258


In [5]:
# data_top200[(data_top200['Artist'].isna()) | 
#             (data_top200['Track Name'].isna()) | 
#             (data_top200['Track URL'].isna())]

In [6]:
data1 = data_top200.drop_duplicates().dropna()
# data1 = data1[data1['Country'] == 'United States']
print(len(data1.index))

8466353


In [7]:
data1.groupby(['Country'])[['Position']].describe()

Position                                                    \
                   count        mean        std  min    25%    50%     75%   
Country                                                                      
Argentina       181786.0  100.504302  57.733307  1.0  51.00  101.0  151.00   
Australia       181790.0  100.503488  57.733798  1.0  51.00  101.0  151.00   
Austria         181325.0  100.494730  57.737344  1.0  51.00  100.0  150.00   
Belgium           5400.0  100.500000  57.739652  1.0  50.75  100.5  150.25   
Bolivia         131118.0   78.727253  50.903239  1.0  37.00   73.0  114.00   
...                  ...         ...        ...  ...    ...    ...     ...   
Turkey          181792.0  100.502915  57.733898  1.0  51.00  101.0  151.00   
United Kingdom  181790.0  100.502173  57.734416  1.0  51.00  101.0  151.00   
United States   181784.0  100.500468  57.733728  1.0  51.00  100.5  150.00   
Uruguay           5389.0  100.307107  57.640128  1.0  50.00  100.0  150.00   
Viet Nam         67242.0   78.674564  51.821415  1.0  36.00   72.0  113.00   

                       
                  max  
Country                
Argentina       200.0  
Australia       200.0  
Austria         200.0  
Belgium         200.0  
Bolivia         200.0  
...               ...  
Turkey          200.0  
United Kingdom  200.0  
United States   200.0  
Uruguay         200.0  
Viet Nam        200.0  

[63 rows x 8 columns]

In [8]:
data2 = data1.sort_values(by=['Country', 'Artist', 'Date', 'Track Name'], ascending=[True,True,True,True])

In [17]:
data3 = data2
data3['Song_min_date'] = data3.groupby(['Country', 'Artist', 'Track Name'])[['Date']].transform('min')

# Calculate the number of days between the current chart day and the song's first day on the chart 
data3['Song_days_since_first'] = data3['Date'] - data3['Song_min_date']
data3['Song_days_since_first'] = data3['Song_days_since_first'].apply(lambda x: x.days)

# Calculate the actual number of days that the song has appeared on the chart 
data3['Song_days_onchart'] = data3.groupby(['Country', 'Artist', 'Track Name']).cumcount() + 1
# The two previous variables are similar, but may differ if the song temporarily goes off the chart and gets back on at a later date

In [18]:
# Identify the first time an artist appears on the Top 200 chart (in a country)
# This needs to be run with the complete dataset, not just on songs with collaboration
data3['Artist_min_date'] = data3.groupby(['Country', 'Artist'])[['Date']].transform('min')

# Generate a dummy variable that = 1 if this is the first time a song appears
data3['Artist_new_song'] = np.where(data3['Song_days_onchart']==1, 1, 0)

In [19]:
data3['Date_lag'] =  data3.groupby(['Country', 'Artist', 'Track Name'])[['Date']].shift(1)

# Calculate the number of days elapsed between a song's current appearance and its last appearance
# If Date_diff = 1, then the song appeared yesterday and today.

data3['Date_diff'] = data3['Date'] - data3['Date_lag']
data3['Date_diff'] = data3['Date_diff'].apply(lambda x: x.days)

In [20]:
data4 = data3

# When Date_diff is > 1 , we assume that the song has temporarily dropped off the chart (hence ending its last consecutive streak),
# so we create a binary variable "Song_new_streak" that signals the beginnning of a new streak
data4['Song_new_streak'] = np.where((data4['Song_days_onchart']==1)|(data4['Date_diff']>1), 1, 0)

# For each country-artist-song's consecutive streak, assign a unique id by performing a cumulative sum of the 'Song_new_streak' variable
data4['Song_streak_id'] = data4.groupby(['Country', 'Artist', 'Track Name'])[['Song_new_streak']].transform('cumsum')

# For each day within a streak, assign a unique id
data4['Song_consec_day'] = data4.groupby(['Country', 'Artist', 'Track Name', 'Song_streak_id']).cumcount() + 1

In [21]:
# Calculate the song's cumulative total number of streams and cumulative average number of streams
data4['Song_cumu_streams'] = data4.groupby(['Country', 'Artist', 'Track Name'])[['Streams']].transform('cumsum')
data4['Song_cumu_mean_streams'] = data4['Song_cumu_streams'] / data4['Song_days_onchart']

In [22]:
data5 = data4

# Calculate an artist's cumulative number of songs that have appeared on the chart
data5['Artist_cumsum_songs'] = data5.groupby(['Country', 'Artist'])[['Artist_new_song']].transform('cumsum')
data5['Artist_cumu_songs'] = data5.groupby(['Country', 'Artist', 'Date'])[['Artist_cumsum_songs']].transform('max')

In [23]:
# Calculate an artist's cumulative number of days of appearing on the chart (not necessarily consecutively)
artist_days = data5.loc[:, ['Country', 'Artist', 'Date']]\
.sort_values(by=['Country', 'Artist', 'Date'], ascending=[True,True,True]).drop_duplicates()
artist_days['Artist_days_onchart'] = artist_days.groupby(['Country', 'Artist']).cumcount() + 1

artist_days

,Country,Artist,Date,Artist_days_onchart
9070494,Argentina,#TocoParaVos,2017-01-01,1
9103053,Argentina,#TocoParaVos,2017-01-02,2
11760080,Argentina,#TocoParaVos,2017-01-03,3
8007416,Argentina,#TocoParaVos,2017-01-04,4
9047990,Argentina,#TocoParaVos,2017-01-05,5
...,...,...,...,...
12598829,Viet Nam,Đức Phúc,2019-06-30,250
8244771,Viet Nam,Đức Phúc,2019-07-01,251
8563289,Viet Nam,하숙자,2018-07-24,1
9668693,Viet Nam,하숙자,2018-07-25,2


In [25]:
data6 = data5.merge(artist_days, how='left', on=['Country', 'Artist', 'Date'])

# Calculate an artist's cumulative total number of streams and cumulative average number of streams
data6['Artist_cumsum_streams'] = data6.groupby(['Country', 'Artist'])[['Streams']].transform('cumsum')
data6['Artist_cumu_streams'] = data6.groupby(['Country', 'Artist', 'Date'])[['Artist_cumsum_streams']].transform('max')

# Needs more reflection for this variable - cumulative averages are not accurate because we only observe the song if it's in the top 200
# if the song is not in the top 200, then the cumulative number of streams cannot be updated
# data6['Artist_cumu_mean_streams'] = data6['Artist_cumu_streams'] / data6['Artist_days_onchart']

In [32]:
# Generate a unique id for each week and year-week 
# (this will be used to merge with other datasets that only update on a weekly basis, e.g. Billboard)
data6['Week_id'] = data6['Date'].dt.strftime('%W')
data6['Year_week_id'] = data6['Date'].dt.strftime('%Y-%W')
data6['Day_week'] = data6['Date'].dt.strftime('%a')

# Create a separate week id to merge with Billboard data, which resets on a Tuesday instead of Monday
data6['Week_id_BB'] = np.where(data6['Day_week'] == 'Mon', 
                               data6['Week_id'].astype(int) - 1, data6['Week_id'].astype(int))

# Calculate the mean number of streams for each week id in a country
data6['Week_mean_streams'] = data6.groupby(['Country', 'Year_week_id'])[['Streams']].transform('mean')

In [ ]:
data6.head()

In [33]:
data6.loc[(data6['Artist']=='Drake') &
          (data6['Country']=='United States') &
          (data6['Track Name']=='One Dance')
          ,:]\
.drop(['Track URL'], axis=1).head()

,Date,Position,Track Name,Artist,Streams,Country,Unnamed: 0,Song_min_date,Song_days_since_first,Song_days_onchart,Artist_min_date,Artist_new_song,Date_lag,Date_diff,Song_new_streak,Song_streak_id,Song_consec_day,Song_cumu_streams,Song_cumu_mean_streams,Artist_cumu_songs,Artist_cumsum_songs,Artist_days_onchart,Artist_cumsum_streams,Artist_cumu_streams,Week_id,Year_week_id,Day_week,Week_mean_streams,Week_id_BB
8253000,2017-01-01,7,One Dance,Drake,753150,United States,7354560.0,2017-01-01,0,1,2017-01-01,1,NaT,NaN,1,1,1,753150,753150.000000,9,5,1,2778807,3909721,00,2017-00,Sun,275695.290000,0
8253010,2017-01-02,13,One Dance,Drake,487243,United States,116825.0,2017-01-01,1,2,2017-01-01,0,2017-01-01,1.0,0,1,2,1240393,620196.500000,10,10,2,6293003,7296288,01,2017-01,Mon,272030.682857,0
8253020,2017-01-03,18,One Dance,Drake,512037,United States,2729475.0,2017-01-01,2,3,2017-01-01,0,2017-01-02,1.0,0,1,3,1752430,584143.333333,10,10,3,9856908,10974059,01,2017-01,Tue,272030.682857,1
8253030,2017-01-04,20,One Dance,Drake,498653,United States,6468062.0,2017-01-01,3,4,2017-01-01,0,2017-01-03,1.0,0,1,4,2251083,562770.750000,10,10,4,13524121,14640932,01,2017-01,Wed,272030.682857,1
8253040,2017-01-05,19,One Dance,Drake,504230,United States,3582297.0,2017-01-01,4,5,2017-01-01,0,2017-01-04,1.0,0,1,5,2755313,551062.600000,10,10,5,17232069,18405690,01,2017-01,Thu,272030.682857,1


In [34]:
data6.loc[(data6['Artist']=='Drake') &
          (data6['Country']=='United States')
          ,:]\
.drop(['Track URL'], axis=1).head(25)

,Date,Position,Track Name,Artist,Streams,Country,Unnamed: 0,Song_min_date,Song_days_since_first,Song_days_onchart,Artist_min_date,Artist_new_song,Date_lag,Date_diff,Song_new_streak,Song_streak_id,Song_consec_day,Song_cumu_streams,Song_cumu_mean_streams,Artist_cumu_songs,Artist_cumsum_songs,Artist_days_onchart,Artist_cumsum_streams,Artist_cumu_streams,Week_id,Year_week_id,Day_week,Week_mean_streams,Week_id_BB
8252996,2017-01-01,35,Controlla,Drake,365140,United States,2567298.0,2017-01-01,0,1,2017-01-01,1,NaT,NaN,1,1,1,365140,3.651400e+05,9,1,1,365140,3909721,00,2017-00,Sun,275695.290000,0
8252997,2017-01-01,2,Fake Love,Drake,1180074,United States,7202613.0,2017-01-01,0,1,2017-01-01,1,NaT,NaN,1,1,1,1180074,1.180074e+06,9,2,1,1545214,3909721,00,2017-00,Sun,275695.290000,0
8252998,2017-01-01,129,Hotline Bling,Drake,183047,United States,2139014.0,2017-01-01,0,1,2017-01-01,1,NaT,NaN,1,1,1,183047,1.830470e+05,9,3,1,1728261,3909721,00,2017-00,Sun,275695.290000,0
8252999,2017-01-01,57,Jumpman,Drake,297396,United States,3843367.0,2017-01-01,0,1,2017-01-01,1,NaT,NaN,1,1,1,297396,2.973960e+05,9,4,1,2025657,3909721,00,2017-00,Sun,275695.290000,0
8253000,2017-01-01,7,One Dance,Drake,753150,United States,7354560.0,2017-01-01,0,1,2017-01-01,1,NaT,NaN,1,1,1,753150,7.531500e+05,9,5,1,2778807,3909721,00,2017-00,Sun,275695.290000,0
8253001,2017-01-01,122,Pop Style,Drake,184790,United States,2033536.0,2017-01-01,0,1,2017-01-01,1,NaT,NaN,1,1,1,184790,1.847900e+05,9,6,1,2963597,3909721,00,2017-00,Sun,275695.290000,0
8253002,2017-01-01,28,Sneakin’,Drake,419434,United States,226710.0,2017-01-01,0,1,2017-01-01,1,NaT,NaN,1,1,1,419434,4.194340e+05,9,7,1,3383031,3909721,00,2017-00,Sun,275695.290000,0
8253003,2017-01-01,170,Still Here,Drake,158154,United States,4454342.0,2017-01-01,0,1,2017-01-01,1,NaT,NaN,1,1,1,158154,1.581540e+05,9,8,1,3541185,3909721,00,2017-00,Sun,275695.290000,0
8253004,2017-01-01,34,Too Good,Drake,368536,United States,7174780.0,2017-01-01,0,1,2017-01-01,1,NaT,NaN,1,1,1,368536,3.685360e+05,9,9,1,3909721,3909721,00,2017-00,Sun,275695.290000,0
8253005,2017-01-02,187,Childs Play,Drake,130083,United States,1351898.0,2017-01-02,0,1,2017-01-01,1,NaT,NaN,1,1,1,130083,1.300830e+05,10,10,2,4039804,7296288,01,2017-01,Mon,272030.682857,0


In [ ]:
# data3.to_csv('2020.12.14 top200_clean.csv')

In [35]:
data3[data3['Country']=='United States'].to_csv('2020.12.14 top200_usa_clean.csv')